<a href="https://colab.research.google.com/github/kirank981/Project_space/blob/main/project_space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing dependences


Install the necessary packages for PyTorch (torch and torchvision) and Flower (flwr) and pandas

In [ ]:
!pip install -q flwr[simulation] torch torchvision matplotlib pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 13.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 39.0 MB/s eta 0:00:00


Import everything we need

In [ ]:
from collections import OrderedDict
from typing import List, Tuple

from google.colab import drive
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
# from torchvision.datasets import CIFAR10



from sklearn.model_selection import train_test_split  # Import the train_test_split function
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

!pip install catboost
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# from tensorflow.keras.layers import Conv2D, Multiply, Input
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense, Multiply

import flwr as fl
from flwr.common import Metrics

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.0 MB/s eta 0:00:00
Training on cpu using PyTorch 2.0.1+cu118 and Flower 1.4.0


# Loading the data

Mounting drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Setting the path to the location of the file

In [ ]:
# Define the path to daily dataset folder
daily_dataset_path = Path('/content/drive/MyDrive/Federated learning implementation/dataset/dataset_archive/daily_dataset/daily_dataset')

# Define the path to daily dataset folder
weather_daily_dataset_path = Path('/content/drive/MyDrive/Federated learning implementation/dataset/dataset_archive/weather_daily_dataset.csv')


## Loading daily data
(of energy consumption)


In [ ]:

# Initializing list to store dataframes
dfs = []

# Loop through the CSV files and reading them into dataframes
for i in range(2):
    filename = f'block_{i}.csv'
    df = pd.read_csv(daily_dataset_path / filename)
    dfs.append(df)

# Concatenating all the dataframes into a single dataframe
energy_daily_data = pd.concat(dfs, ignore_index=True)

# NRATM(Not Required At The Moment)
# # Group the data by LCLid and create a dictionary of dataframes
# grouped_data = dict(tuple(energy_daily_data.groupby('LCLid')))


Loading data using file name

In [ ]:
# Loading data from a specific CSV file
specific_file_data = pd.read_csv(daily_dataset_path / 'block_0.csv')

# Displaying data
print(specific_file_data)

           LCLid         day  energy_median  energy_mean  energy_max  \
0      MAC000002  2012-10-12         0.1385     0.154304       0.886   
1      MAC000002  2012-10-13         0.1800     0.230979       0.933   
2      MAC000002  2012-10-14         0.1580     0.275479       1.085   
3      MAC000002  2012-10-15         0.1310     0.213688       1.164   
4      MAC000002  2012-10-16         0.1450     0.203521       0.991   
...          ...         ...            ...          ...         ...   
25569  MAC005492  2014-02-24         0.1690     0.175042       0.378   
25570  MAC005492  2014-02-25         0.1550     0.160792       0.545   
25571  MAC005492  2014-02-26         0.1490     0.178542       0.687   
25572  MAC005492  2014-02-27         0.1140     0.146167       0.478   
25573  MAC005492  2014-02-28         0.0880     0.088000       0.088   

       energy_count  energy_std  energy_sum  energy_min  
0                46    0.196034       7.098       0.000  
1                48

### Loading required data
Creating a DataFrame that have only the required data

In [ ]:
selected_column = ['LCLid','energy_sum','day']
energy_daily_selected=energy_daily_data[selected_column]
# print(energy_daily_selected)

# Group the data by LCLid and create a dictionary of dataframes, allowing to access each dataframe separately using the LCLid as the key
grouped_data_selected = dict(tuple(energy_daily_selected.groupby('LCLid')))
# Display the data for 'MAC000002'
print('\n')
print('Data report of MAC000002')
print(grouped_data_selected['MAC000002'])




Data report of MAC000002
         LCLid  energy_sum         day
0    MAC000002       7.098  2012-10-12
1    MAC000002      11.087  2012-10-13
2    MAC000002      13.223  2012-10-14
3    MAC000002      10.257  2012-10-15
4    MAC000002       9.769  2012-10-16
..         ...         ...         ...
500  MAC000002      12.528  2014-02-24
501  MAC000002      11.826  2014-02-25
502  MAC000002      12.328  2014-02-26
503  MAC000002      20.518  2014-02-27
504  MAC000002       1.387  2014-02-28

[505 rows x 3 columns]


## Loading daily weather data

Creating a 'day' column that stores only the date values from 'time' column
(for linking weather dataset 'day' with daily dataset 'day')

In [ ]:
# Load the weather dataset into a DataFrame
weather_daily_data = pd.read_csv(weather_daily_dataset_path)

# Convert the 'time' column to datetime format
weather_daily_data['time'] = pd.to_datetime(weather_daily_data['time'])

# Calculate the mean temperature for each day and store it in a new column 'mean_temp'
weather_daily_data['mean_temp'] = (weather_daily_data['temperatureMax'] + weather_daily_data['temperatureMin']) / 2

# Print the updated DataFrame
print(weather_daily_data)

     temperatureMax   temperatureMaxTime  windBearing                 icon  \
0             11.96  2011-11-11 23:00:00          123                  fog   
1              8.59  2011-12-11 14:00:00          198    partly-cloudy-day   
2             10.33  2011-12-27 02:00:00          225    partly-cloudy-day   
3              8.07  2011-12-02 23:00:00          232                 wind   
4              8.22  2011-12-24 23:00:00          252  partly-cloudy-night   
..              ...                  ...          ...                  ...   
877            9.03  2014-01-26 16:00:00          233    partly-cloudy-day   
878           10.31  2014-02-27 14:00:00          224    partly-cloudy-day   
879           18.97  2014-03-09 14:00:00          172  partly-cloudy-night   
880            8.83  2014-02-12 16:00:00          210                 wind   
881            9.90  2014-02-15 12:00:00          233                 wind   

     dewPoint   temperatureMinTime  cloudCover  windSpeed  pres

### Loading required data
Creating a DataFrame that have only the required data

In [ ]:
# Create a new DataFrame with selected columns
selected_columns = ['mean_temp', 'pressure', 'humidity', 'windSpeed', 'time']
weather_selected = weather_daily_data[selected_columns]

# Print the new dataset
print(weather_selected)

     mean_temp  pressure  humidity  windSpeed       time
0       10.405   1016.08      0.95       3.88 2011-11-11
1        5.535   1007.71      0.88       3.94 2011-12-11
2        9.180   1032.76      0.74       3.54 2011-12-27
3        5.315   1012.12      0.87       3.00 2011-12-02
4        5.695   1028.17      0.80       4.46 2011-12-24
..         ...       ...       ...        ...        ...
877      6.145   1002.10      0.79       4.55 2014-01-26
878      7.120   1007.02      0.74       4.14 2014-02-27
879     13.310   1022.44      0.58       2.78 2014-03-09
880      5.930    994.27      0.75       7.24 2014-02-12
881      7.640    988.63      0.69       9.96 2014-02-15

[882 rows x 5 columns]


# Dataset with Household energy consumption values and weather values

## For all households

In [ ]:
# Convert 'time' column in weather_selected to datetime objects
weather_selected['time'] = pd.to_datetime(weather_selected['time'])

# Create a list to store the merged data DataFrames
merged_data_list = []

# Iterate through each LCLid in energy_daily_selected
for lclid, data in grouped_data_selected.items():
    # Convert 'day' column in current LCLid data to datetime objects
    data['day'] = pd.to_datetime(data['day'])

    # Merge the current LCLid data with weather_selected based on the common date values
    merged_data_lclid = pd.merge(weather_selected, data, left_on='time', right_on='day', how='inner')

    # Drop the redundant 'day' column from the merged data
    merged_data_lclid.drop(columns=['day'], inplace=True)

    # Append the merged data to the merged_data_list
    merged_data_list.append(merged_data_lclid)

# Concatenate the merged data DataFrames in the list
merged_data = pd.concat(merged_data_list, ignore_index=True)

# Display the merged dataset
print(merged_data)


<ipython-input-10-fc90630300b6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_selected['time'] = pd.to_datetime(weather_selected['time'])


       mean_temp  pressure  humidity  windSpeed       time      LCLid  \
0         10.000   1001.79      0.81       6.54 2012-11-25  MAC000002   
1         12.170   1008.74      0.90       5.74 2012-11-20  MAC000002   
2          7.830    979.63      0.85       4.07 2012-11-01  MAC000002   
3          5.295   1020.29      0.87       3.58 2012-11-06  MAC000002   
4          4.650   1001.72      0.80       5.63 2012-12-07  MAC000002   
...          ...       ...       ...        ...        ...        ...   
26732      4.470   1001.76      0.91       1.52 2014-01-30  MAC005492   
26733      6.145   1002.10      0.79       4.55 2014-01-26  MAC005492   
26734      7.120   1007.02      0.74       4.14 2014-02-27  MAC005492   
26735      5.930    994.27      0.75       7.24 2014-02-12  MAC005492   
26736      7.640    988.63      0.69       9.96 2014-02-15  MAC005492   

       energy_sum  
0          10.545  
1          11.221  
2          12.209  
3          11.663  
4          13.248  
...

## For one household

In [ ]:
print(merged_data[merged_data['LCLid'] == "MAC003719"])

       mean_temp  pressure  humidity  windSpeed       time      LCLid  \
21058     10.000   1001.79      0.81       6.54 2012-11-25  MAC003719   
21059     12.170   1008.74      0.90       5.74 2012-11-20  MAC003719   
21060      7.830    979.63      0.85       4.07 2012-11-01  MAC003719   
21061      5.295   1020.29      0.87       3.58 2012-11-06  MAC003719   
21062      4.650   1001.72      0.80       5.63 2012-12-07  MAC003719   
...          ...       ...       ...        ...        ...        ...   
21331      4.470   1001.76      0.91       1.52 2014-01-30  MAC003719   
21332      6.145   1002.10      0.79       4.55 2014-01-26  MAC003719   
21333      7.120   1007.02      0.74       4.14 2014-02-27  MAC003719   
21334      5.930    994.27      0.75       7.24 2014-02-12  MAC003719   
21335      7.640    988.63      0.69       9.96 2014-02-15  MAC003719   

       energy_sum  
21058      13.952  
21059      12.864  
21060      18.794  
21061      16.750  
21062      15.547  
...

# Splitting dataset to training and testing sets



## Preprocessing dataset

### Counting amount of data for each LCLid (data available for each household)

In [ ]:
# Convert 'time' column in weather_selected to datetime objects
weather_selected['time'] = pd.to_datetime(weather_selected['time'])

# Merge the data into merged_data DataFrame as described in your previous code

# Count the number of data rows for each LCLid
lclid_data_counts = merged_data['LCLid'].value_counts()

# Display the counts for each LCLid
print("Data row counts for each LCLid:")
print(lclid_data_counts)


Data row counts for each LCLid:
MAC000246    392
MAC004529    374
MAC000323    298
MAC000386    291
MAC004387    288
            ... 
MAC003463    154
MAC004034    154
MAC003718    154
MAC002613    149
MAC001074     53
Name: LCLid, Length: 100, dtype: int64


<ipython-input-12-467dbfca5fe4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_selected['time'] = pd.to_datetime(weather_selected['time'])


### Identifiting the no of households with insufficient amount of data

In [ ]:
# Count the number of data rows for each LCLid
lclid_data_counts = merged_data['LCLid'].value_counts()


# Get the total number of unique LCLid values
total_lclids = len(lclid_data_counts)

# Count the number of LCLid values with less than 100 data rows
count_less_than_100 = (lclid_data_counts < 100).sum()

# Display the count of LCLid values with less than 100 data rows
print("Number of LCLid values with less than 100 data rows:", count_less_than_100)

# Display the total number of unique LCLid values
print("Total number of unique LCLid values:", total_lclids)

Number of LCLid values with less than 100 data rows: 1
Total number of unique LCLid values: 100


### Removing the households with less data, from the dataset

In [ ]:
# Get the list of LCLid values with less than 100 data rows
lclids_to_remove = lclid_data_counts[lclid_data_counts < 100].index

# Remove rows corresponding to LCLid values with less than 100 data rows
filtered_data = merged_data[~merged_data['LCLid'].isin(lclids_to_remove)]

# also removing rows with NaN values in the dataset
filtered_data = filtered_data.dropna()

# Display the filtered data
print(filtered_data)

       mean_temp  pressure  humidity  windSpeed       time      LCLid  \
0         10.000   1001.79      0.81       6.54 2012-11-25  MAC000002   
1         12.170   1008.74      0.90       5.74 2012-11-20  MAC000002   
2          7.830    979.63      0.85       4.07 2012-11-01  MAC000002   
3          5.295   1020.29      0.87       3.58 2012-11-06  MAC000002   
4          4.650   1001.72      0.80       5.63 2012-12-07  MAC000002   
...          ...       ...       ...        ...        ...        ...   
26732      4.470   1001.76      0.91       1.52 2014-01-30  MAC005492   
26733      6.145   1002.10      0.79       4.55 2014-01-26  MAC005492   
26734      7.120   1007.02      0.74       4.14 2014-02-27  MAC005492   
26735      5.930    994.27      0.75       7.24 2014-02-12  MAC005492   
26736      7.640    988.63      0.69       9.96 2014-02-15  MAC005492   

       energy_sum  
0          10.545  
1          11.221  
2          12.209  
3          11.663  
4          13.248  
...

## Splitting the dataset to train and test
 where the split ratio(70:30) is made with every househould's data.

In [45]:
# Define the split percentages
train_percentage = 0.7 # 70% for training, 30% for testing
min_data_points = 10  # Minimum number of data points required for an LCLid

# Create a list to store DataFrames for training and testing
train_data_list = []
test_data_list = []

# Iterate through each unique LCLid and split the data based on train_percentage
unique_lclids = filtered_data['LCLid'].unique()
for lclid in unique_lclids:
    lclid_data = filtered_data[filtered_data['LCLid'] == lclid]

    # Check if there are sufficient data points for the current LCLid
    if len(lclid_data) >= min_data_points:
        # Split the data for the current LCLid into training and test sets
        train_data_lclid, test_data_lclid = train_test_split(lclid_data, train_size=train_percentage, shuffle=False)

        # Randomize the rows within each subset
        train_data_lclid = train_data_lclid.sample(frac=1, random_state=42)
        test_data_lclid = test_data_lclid.sample(frac=1, random_state=42)

        # Append to the train_data_list and test_data_list
        train_data_list.append(train_data_lclid)
        test_data_list.append(test_data_lclid)

# Concatenate the DataFrames in the lists
FL_train_set = pd.concat(train_data_list, ignore_index=True)
FL_test_set = pd.concat(test_data_list, ignore_index=True)

# Store the training and test sets in separate lists
FL_train_set_list = train_data_list
FL_test_sets_list = test_data_list

# Display the training and test sets
print("Training set:")
print(FL_train_set)
print("Test set:")
print(FL_test_set)

Training set:
       mean_temp  pressure  humidity  windSpeed       time      LCLid  \
0          5.170   1006.05      0.90       1.90 2013-11-09  MAC002068   
1          4.965   1006.45      0.86       3.61 2013-01-26  MAC002068   
2          7.360   1001.69      0.86       3.79 2012-12-17  MAC002068   
3          9.700   1010.05      0.80       3.17 2012-11-09  MAC002068   
4          9.040    991.49      0.86       4.93 2012-12-15  MAC002068   
...          ...       ...       ...        ...        ...        ...   
18614      9.940   1019.67      0.85       3.44 2013-12-13  MAC003686   
18615      7.195   1005.46      0.84       4.09 2012-12-27  MAC003686   
18616      9.020   1020.10      0.80       3.67 2013-10-30  MAC003686   
18617      3.565    991.02      0.91       0.73 2013-03-18  MAC003686   
18618      7.450   1024.32      0.86       4.20 2012-11-07  MAC003686   

       energy_sum  
0          14.117  
1          11.782  
2          12.446  
3          24.090  
4        

### For FL training, validation and test sets

In [46]:
# Display the first training set
print("First Training set:")
print(FL_train_set_list[0])
print("First Test set:")
print(FL_test_sets_list[0])

First Training set:
      mean_temp  pressure  humidity  windSpeed       time      LCLid  \
9490      5.170   1006.05      0.90       1.90 2013-11-09  MAC002068   
9614      4.965   1006.45      0.86       3.61 2013-01-26  MAC002068   
9453      7.360   1001.69      0.86       3.79 2012-12-17  MAC002068   
9445      9.700   1010.05      0.80       3.17 2012-11-09  MAC002068   
9448      9.040    991.49      0.86       4.93 2012-12-15  MAC002068   
...         ...       ...       ...        ...        ...        ...   
9588      7.380    991.64      0.82       2.94 2013-11-04  MAC002068   
9589      5.565   1013.74      0.78       3.66 2013-03-04  MAC002068   
9557      1.265   1011.47      0.88       2.08 2013-01-14  MAC002068   
9451      6.775   1010.19      0.97       3.43 2012-11-24  MAC002068   
9421     10.000   1001.79      0.81       6.54 2012-11-25  MAC002068   

      energy_sum  
9490      14.117  
9614      11.782  
9453      12.446  
9445      24.090  
9448      34.729  
.

### Centralised training, validation and test sets

In [50]:

# Define the split percentages
train_percentage = 0.6  # 60% for training

# Concatenate the DataFrames in the lists
FL_train_set = pd.concat(train_data_list, ignore_index=True)
FL_test_set = pd.concat(test_data_list, ignore_index=True)

# Randomize the rows of FL_train_set and FL_test_set
FL_train_set = FL_train_set.sample(frac=1, random_state=42)
FL_test_set = FL_test_set.sample(frac=1, random_state=42)

# Split FL_train_set into Central_train_data and Central_val_data
Central_train_data, Central_val_data = train_test_split(FL_train_set, train_size=train_percentage, random_state=42)

# Central_test_data is FL_test_set
Central_test_data = FL_test_set.copy()

# Display the sizes of the sets
print("Central Training set size:", len(Central_train_data))
print("Central Validation set size:", len(Central_val_data))
print("Central Test set size:", len(Central_test_data))


Central Training set size: 11171
Central Validation set size: 7448
Central Test set size: 8065


Displaying training, validation and test sets

In [51]:
# Display the training and test sets
print("Training set:")
print(Central_train_data)
print("Validation set:")
print(Central_val_data)
print("Test set:")
print(Central_test_data)


Training set:
       mean_temp  pressure  humidity  windSpeed       time      LCLid  \
12318      2.825   1015.57      0.67       1.81 2013-03-14  MAC002601   
15419      8.990   1034.64      0.87       1.23 2013-11-28  MAC001734   
17920      2.295   1015.80      0.94       1.55 2013-01-10  MAC001776   
14374     12.675    998.60      0.82       4.55 2013-11-02  MAC001628   
9251       7.440   1009.77      0.73       5.66 2013-12-22  MAC001510   
...          ...       ...       ...        ...        ...        ...   
16810      5.515   1004.26      0.78       4.83 2013-11-21  MAC000323   
4487       9.495   1011.94      0.85       4.15 2014-01-08  MAC003680   
9839       4.140   1014.89      0.84       3.07 2013-12-29  MAC004179   
17551      8.145   1005.53      0.90       2.00 2013-11-08  MAC000974   
2416       8.310   1010.88      0.83       6.58 2013-12-30  MAC003553   

       energy_sum  
12318   36.984000  
15419   35.806000  
17920    3.163000  
14374    5.485000  
9251    3

# Centralised Training

## Defining features and target column

In [52]:
# Separate features and target variable
Central_X_train = Central_train_data[['mean_temp', 'pressure', 'humidity', 'windSpeed']]
Central_y_train = Central_train_data['energy_sum']

# Separate features and target variable
Central_X_val = Central_val_data[['mean_temp', 'pressure', 'humidity', 'windSpeed']]
Central_y_val = Central_val_data['energy_sum']

# Separate features and target variable
Central_X_test = Central_test_data[['mean_temp', 'pressure', 'humidity', 'windSpeed']]
Central_y_test = Central_test_data['energy_sum']


## Preprocessing (scaler and imputer)

In [53]:
# Define the datasets
datasets = [
    (Central_X_train, Central_y_train, "Central Training"),
    (Central_X_val, Central_y_val, "Central Validation"),
    (Central_X_test, Central_y_test, "Central Test")
]

# Preprocess each dataset
for X, y, name in datasets:
    # Impute missing values and scale features
    X_scaled = StandardScaler().fit_transform(SimpleImputer(strategy='mean').fit_transform(X))

    # Display dataset name and shape
    print(f"{name} Dataset:")
    print("Original Shape:", X.shape)
    print("Preprocessed Shape:", X_scaled.shape)
    print()

Central Training Dataset:
Original Shape: (11171, 4)
Preprocessed Shape: (11171, 4)

Central Validation Dataset:
Original Shape: (7448, 4)
Preprocessed Shape: (7448, 4)

Central Test Dataset:
Original Shape: (8065, 4)
Preprocessed Shape: (8065, 4)



In [ ]:
# Check for missing values
print(Central_y_train.isnull().sum())
print(Central_X_train.isnull().sum())

# Replace missing values with mean (you can choose another strategy)
Central_y_train.fillna(Central_y_train.mean(), inplace=True)
Central_X_train.fillna(Central_X_train.mean(), inplace=True)

# Check for missing values
print(Central_y_train.isnull().sum())
print(Central_X_train.isnull().sum())

0
mean_temp    0
pressure     0
humidity     0
windSpeed    0
dtype: int64
0
mean_temp    0
pressure     0
humidity     0
windSpeed    0
dtype: int64


<ipython-input-21-e51efdbfd96c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Central_X_train.fillna(Central_X_train.mean(), inplace=True)
<ipython-input-21-e51efdbfd96c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Central_X_train.fillna(Central_X_train.mean(), inplace=True)
<ipython-input-21-e51efdbfd96c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

## Model selection

In [54]:

# # List of models to train
# models = [
#     LinearRegression(),
#     DecisionTreeRegressor(),
#     RandomForestRegressor(),
#     GradientBoostingRegressor(),
#     XGBRegressor(),
#     LGBMRegressor(),
#     CatBoostRegressor(),
#     SVR(),
#     MLPRegressor(),
#     KNeighborsRegressor()
# ]

# # Dictionary to store model performances
# model_performances = {}

# # Train and evaluate each model on the validation set
# for model in models:
#     model_name = model.__class__.__name__
#     print(f"Training {model_name}...")
#     model.fit(Central_X_train, Central_y_train)

#     # Predict on the validation set
#     y_pred_val = model.predict(Central_X_val)
#     # print(y_pred_val)

#     # Calculate Mean Squared Error
#     # mse_val = mean_squared_error(Central_y_val, y_pred_val)
#     model_performances[model_name] = (Central_y_val-y_pred_val)
#     print(model_performances[model_name])

# # Find the best-performing model
# best_model = min(model_performances, key=model_performances.get)
# print(f"Best-performing model: {best_model} (MSE: {model_performances[best_model]})")


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define models
linear_model = LinearRegression()
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
xgb_model = xgb.XGBRegressor(n_estimators=100, objective='reg:squarederror', random_state=0)

# Remove rows with missing values from both features and target variables
Central_train_data_cleaned = Central_train_data.dropna()
Central_X_train = Central_train_data_cleaned[['mean_temp', 'pressure', 'humidity', 'windSpeed']]
Central_y_train = Central_train_data_cleaned['energy_sum']

# Drop rows with NaN values in the target variable in the validation set
Central_val_data.dropna(subset=['energy_sum'], inplace=True)

# Train linear model
linear_model.fit(Central_X_train, Central_y_train)

# Train random forest model
rf_model.fit(Central_X_train, Central_y_train)

# Train XGBoost model
xgb_model.fit(Central_X_train, Central_y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=0, ...)

In [ ]:
# Build a simple feedforward neural network(FNN)
fnn_model = keras.Sequential([
    layers.Input(shape=(4,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
])

fnn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
fnn_model.fit(Central_X_train, Central_y_train, epochs=10, batch_size=32, validation_data=(Central_X_val, Central_y_val))

Epoch 1/10
501/501 [==============================] - 8s 4ms/step - loss: 7357.3325 - mse: 7357.3325 - val_loss: 434.7622 - val_mse: 434.7622
Epoch 2/10
501/501 [==============================] - 2s 4ms/step - loss: 420.3943 - mse: 420.3943 - val_loss: 437.6509 - val_mse: 437.6509
Epoch 3/10
501/501 [==============================] - 2s 4ms/step - loss: 421.2672 - mse: 421.2672 - val_loss: 437.2649 - val_mse: 437.2649
Epoch 4/10
501/501 [==============================] - 2s 5ms/step - loss: 422.0505 - mse: 422.0505 - val_loss: 435.8417 - val_mse: 435.8417
Epoch 5/10
501/501 [==============================] - 3s 5ms/step - loss: 421.1493 - mse: 421.1493 - val_loss: 434.3184 - val_mse: 434.3184
Epoch 6/10
501/501 [==============================] - 2s 5ms/step - loss: 421.1183 - mse: 421.1183 - val_loss: 448.0761 - val_mse: 448.0761
Epoch 7/10
501/501 [==============================] - 2s 4ms/step - loss: 421.0824 - mse: 421.0824 - val_loss: 433.8567 - val_mse: 433.8567
Epoch 8/10
501/501

In [ ]:
# Building a simple RNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

# Create an RNN model
simple_rnn_model = Sequential()
simple_rnn_model.add(SimpleRNN(64, activation='relu', input_shape=(Central_X_train.shape[1], 1)))
simple_rnn_model.add(Dense(1))

# Compile the model
simple_rnn_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

# Train the model
simple_rnn_model.fit(Central_X_train, Central_y_train, epochs=10, batch_size=32, validation_data=(Central_X_val, Central_y_val))


Epoch 1/10
501/501 [==============================] - 7s 10ms/step - loss: 466.6749 - mse: 466.6749 - val_loss: 434.1538 - val_mse: 434.1538
Epoch 2/10
501/501 [==============================] - 3s 6ms/step - loss: 428.7281 - mse: 428.7281 - val_loss: 436.5609 - val_mse: 436.5609
Epoch 3/10
501/501 [==============================] - 4s 7ms/step - loss: 427.6934 - mse: 427.6934 - val_loss: 435.1342 - val_mse: 435.1342
Epoch 4/10
501/501 [==============================] - 4s 9ms/step - loss: 425.3786 - mse: 425.3786 - val_loss: 435.0079 - val_mse: 435.0079
Epoch 5/10
501/501 [==============================] - 6s 11ms/step - loss: 425.2627 - mse: 425.2627 - val_loss: 434.7839 - val_mse: 434.7839
Epoch 6/10
501/501 [==============================] - 5s 9ms/step - loss: 420.9419 - mse: 420.9419 - val_loss: 448.7663 - val_mse: 448.7663
Epoch 7/10
501/501 [==============================] - 3s 7ms/step - loss: 420.8588 - mse: 420.8588 - val_loss: 439.7771 - val_mse: 439.7771
Epoch 8/10
501/501

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Reshape data for LSTM input (samples, timesteps, features)
num_timesteps = 1
num_features = Central_X_train.shape[1]

X_train_reshaped = Central_X_train.values.reshape((Central_X_train.shape[0], num_timesteps, num_features))
X_val_reshaped = Central_X_val.values.reshape((Central_X_val.shape[0], num_timesteps, num_features))

# Create an LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(64, activation='relu', input_shape=(num_timesteps, num_features)))
lstm_model.add(Dense(1))

# Compile the model
lstm_model.compile(loss='mse', optimizer='adam')

# Train the model
lstm_model.fit(X_train_reshaped, Central_y_train, epochs=10, batch_size=32, validation_data=(X_val_reshaped, Central_y_val))


Epoch 1/10
501/501 [==============================] - 5s 6ms/step - loss: 438.3049 - val_loss: 433.8857
Epoch 2/10
501/501 [==============================] - 3s 6ms/step - loss: 421.3251 - val_loss: 444.0783
Epoch 3/10
501/501 [==============================] - 3s 7ms/step - loss: 421.2768 - val_loss: 466.5037
Epoch 4/10
501/501 [==============================] - 5s 10ms/step - loss: 420.6349 - val_loss: 433.2047
Epoch 5/10
501/501 [==============================] - 3s 6ms/step - loss: 421.9791 - val_loss: 441.8762
Epoch 6/10
501/501 [==============================] - 3s 7ms/step - loss: 421.0463 - val_loss: 437.5651
Epoch 7/10
501/501 [==============================] - 3s 6ms/step - loss: 420.1023 - val_loss: 432.5734
Epoch 8/10
501/501 [==============================] - 5s 9ms/step - loss: 420.3820 - val_loss: 432.7023
Epoch 9/10
501/501 [==============================] - 4s 7ms/step - loss: 419.5619 - val_loss: 432.3607
Epoch 10/10
501/501 [==============================] - 3s 6ms/s

### Defining the Evaluation function

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
def evaluate_model(model, X, y):
    predictions = model.predict(X)
    mse = mean_squared_error(y, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, predictions)
    r2 = r2_score(y, predictions)
    print(f"Model: {model.__class__.__name__}")
    print("Mean Squared Error:", mse)
    print("Root Mean Squared Error:", rmse)
    print("Mean Absolute Error:", mae)
    print("R-squared:", r2)
    print("R-squared variance:", r2*100)
    print()


### Using the Evalution Function

In [ ]:
# Define the models as trained earlier
models = [
    ('Linear Regression', linear_model),
    ('Random Forest', rf_model),
    ('XGBoost', xgb_model),
    ('Neural Network', fnn_model),
    ('RNN',simple_rnn_model),
    ('LSTM',lstm_model)
]

# Evaluate each model
for model_name, model in models:
    print(f'\n{model_name}:')
    if model_name == 'LSTM':
        X_val_reshaped = Central_X_val.values.reshape((Central_X_val.shape[0], num_timesteps, num_features))
        evaluate_model(model, X_val_reshaped, Central_y_val)
    else:
        evaluate_model(model, Central_X_val, Central_y_val)




Linear Regression:
Model: LinearRegression
Mean Squared Error: 431.6772158559222
Root Mean Squared Error: 20.7768432601279
Mean Absolute Error: 13.603564854307745
R-squared: 0.0045177866416201695
R-squared variance: 0.45177866416201695


Random Forest:
Model: RandomForestRegressor
Mean Squared Error: 438.707994809373
Root Mean Squared Error: 20.94535735692693
Mean Absolute Error: 13.664868603764097
R-squared: -0.01169575240360654
R-squared variance: -1.169575240360654


XGBoost:
Model: XGBRegressor
Mean Squared Error: 438.3924033014123
Root Mean Squared Error: 20.93782231516478
Mean Absolute Error: 13.669758902878602
R-squared: -0.010967973124732744
R-squared variance: -1.0967973124732744


Neural Network:
167/167 [==============================] - 0s 2ms/step
Model: Sequential
Mean Squared Error: 439.17764744601584
Root Mean Squared Error: 20.95656573596962
Mean Absolute Error: 12.937968727007302
R-squared: -0.012778808977041312
R-squared variance: -1.2778808977041312


RNN:
167/167 

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define a parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

# Create XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=0)

# Create GridSearchCV instance
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)

# Train the model using GridSearchCV
grid_search.fit(Central_X_train, Central_y_train)

# Get the best model from the grid search
best_xgb_model = grid_search.best_estimator_

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.3s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.3s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.2s
[CV] END 

# Should be doing

## Gated Convolution Neural Network (GCNN)

In [ ]:
# class GatedCNN24(nn.Module):
#     def __init__(self, input_channels, hidden_sizes):
#         super(GatedCNN24, self).__init__()
#         self.input_channels = input_channels
#         self.hidden_sizes = hidden_sizes

#         self.conv1 = nn.Conv2d(self.input_channels, self.hidden_sizes[0], kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(self.hidden_sizes[0], self.hidden_sizes[1], kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(self.hidden_sizes[1], self.hidden_sizes[2], kernel_size=3, padding=1)

#         self.gate_conv1 = nn.Conv2d(self.input_channels, self.hidden_sizes[0], kernel_size=3, padding=1)
#         self.gate_conv2 = nn.Conv2d(self.hidden_sizes[0], self.hidden_sizes[1], kernel_size=3, padding=1)
#         self.gate_conv3 = nn.Conv2d(self.hidden_sizes[1], self.hidden_sizes[2], kernel_size=3, padding=1)

#         self.fc1 = nn.Linear(self.hidden_sizes[2], 1)

#     def forward(self, x):
#         # Convolutional layers
#         conv1_out = self.conv1(x)
#         conv2_out = self.conv2(conv1_out)
#         conv3_out = self.conv3(conv2_out)

#         # Gating layers
#         gate_conv1_out = torch.sigmoid(self.gate_conv1(x))
#         gate_conv2_out = torch.sigmoid(self.gate_conv2(conv1_out))
#         gate_conv3_out = torch.sigmoid(self.gate_conv3(conv2_out))

#         # Element-wise multiplication (gating)
#         gated_conv1 = conv1_out * gate_conv1_out
#         gated_conv2 = conv2_out * gate_conv2_out
#         gated_conv3 = conv3_out * gate_conv3_out

#         # Sum along channels
#         gated_sum = gated_conv1 + gated_conv2 + gated_conv3

#         # Global average pooling
#         pooled = torch.mean(gated_sum, dim=[2, 3])

#         # Fully connected layer
#         output = self.fc1(pooled)
#         return output.squeeze()

# class EnergyDataset(Dataset):
#     def __init__(self, features, targets):
#         self.features = features
#         self.targets = targets

#     def __len__(self):
#         return len(self.features)

#     def __getitem__(self, idx):
#         inputs = torch.tensor(self.features.iloc[idx].values, dtype=torch.float32)
#         target = torch.tensor(self.targets.iloc[idx], dtype=torch.float32)
#         return inputs, target

# # Define hyperparameters
# input_channels = 4  # Number of features
# hidden_sizes = [10, 8, 1]
# learning_rate = 0.001
# num_epochs = 10
# batch_size = 50

# # Create DataLoader for training
# train_dataset = EnergyDataset(Central_X_train, Central_y_train)
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# # Create the model
# GatedCNN_model = GatedCNN24(input_channels, hidden_sizes)

# # Define loss function and optimizer
# criterion = nn.MSELoss()
# optimizer = optim.Adam(GatedCNN_model.parameters(), lr=learning_rate)

# # Training loop
# for epoch in range(num_epochs):
#     GatedCNN_model.train()  # Set the model in training mode
#     for inputs, targets in train_loader:
#         optimizer.zero_grad()

#         # Reshape the input tensor to match expected shape [batch_size, channels, height, width]
#         inputs = inputs.view(inputs.size(0), input_channels, 1, 1)

#         outputs = GatedCNN_model(inputs)
#         loss = criterion(outputs, targets)
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


## Gated Neural Network model

In [ ]:
# # Build the Gated Neural Network model
# inputs = Input(shape=(4,))
# dense_layer = Dense(16, activation='relu')(inputs)

# # Gating mechanism
# dense_gate = Dense(16, activation='sigmoid')(inputs)
# gated_dense = Multiply()([dense_layer, dense_gate])

# output = Dense(1, activation='linear')(gated_dense)

# GNN_model = Model(inputs=inputs, outputs=output)

# # Compile the model
# GNN_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

# # Train the model
# GNN_model.fit(Central_X_train, Central_y_train, epochs=10, batch_size=32, validation_split=0.2)

# a dataset, a model, a training function, and a test function.


# Federated learning

Implementing FLower Client

In [60]:
import flwr as fl

from flwr.client import NumPyClient
import xgboost as xgb
from sklearn.metrics import accuracy_score

class XGBoostClient(fl.client.NumPyClient):
    def __init__(self, model, X_train, y_train, X_val, y_val):
        self.model = model
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val

    def get_parameters(self):
        return self.model.get_booster().get_dump()

    def fit(self, parameters, config):
        new_model = xgb.Booster(params=config, model_str=parameters[0])
        self.model = new_model

        self.model.fit(self.X_train, self.y_train)

        return self.model.get_booster().get_dump()

    def evaluate(self, parameters, config):
        new_model = xgb.Booster(params=config, model_str=parameters[0])
        self.model = new_model

        y_pred = self.model.predict(self.X_val)
        accuracy = accuracy_score(self.y_val, y_pred)

        return accuracy



In [67]:
# the best hyperparameters in best_xgb_model
best_hyperparameters = best_xgb_model.get_xgb_params()

# Create an instance of your XGBoost model with the best hyperparameters
best_xgb_model_instance = xgb.XGBRegressor(**best_hyperparameters)


# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Define a function to create a virtual client
def create_virtual_client(lclid):
    client_data = FL_train_set[FL_train_set['LCLid'] == lclid]
    X_client = client_data[['mean_temp', 'pressure', 'humidity', 'windSpeed']]
    y_client = client_data['energy_sum']
    return XGBoostClient(best_xgb_model_instance, X_client, y_client,  FL_test_set[['mean_temp', 'pressure', 'humidity', 'windSpeed']], FL_test_set['energy_sum'])

# Create a list of virtual clients
virtual_clients = [create_virtual_client(lclid) for lclid in unique_lclids]

# Define the client function for Flower simulation
def client_fn(cid: str) -> XGBoostClient:
    virtual_client = virtual_clients[int(cid)]
    return virtual_client

# Define the Flower simulation configuration
config = fl.server.ServerConfig(num_rounds=5)

# Start the Flower simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=len(virtual_clients),
    config=config,
    strategy=fl.server.strategy.FedAvg(),
    client_resources=None,  # Adjust client resources as needed
)


INFO flwr 2023-08-14 20:08:03,665 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-08-14 20:08:07,947	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-08-14 20:08:10,130 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7815023003.0, 'node:__internal_head__': 1.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3907511500.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7815023003.0, 'node:__internal_head__': 1.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3907511500.0}
INFO flwr 2023-08-14 20:08:10,139 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-08-14 20:08:10,143 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters

RayTaskError(TypeError): ignored

federated learning using X